In [0]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import ollama
from IPython.display import Markdown, display, update_display

In [0]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

In [0]:
openai = OpenAI()
claude = anthropic.Anthropic()

In [0]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
ollama_model = "llama3.2"

In [0]:
gpt_system = "You are a knowledgable but sarcastic team lead at a software development company. \
You manage a team with two more junior developers. \
You might come across as aggressive but that's just your humor. "

claude_system = "You are one of the junior developers at a software development company. \
You work in a team of three. \
You are nerdy, introvert but gets the job done efficiently. "

llama_system = "You are one of the junior developers at a software development company. \
You have two other developers in your team.\
You are more talks and less work kind of person. "

gpt_messages = ["Hi, how is it going?"]
claude_messages = ["Hi."]
llama_messages = ["Hey, what's up everyone?"]

In [0]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt_msg, claude_msg, llama_msg in zip(gpt_messages, claude_messages, llama_messages):
        messages.append({"role": "assistant", "content": gpt_msg})
        messages.append({"role": "user", "content": claude_msg})
        messages.append({"role": "user", "content": llama_msg})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [0]:
call_gpt()

In [0]:
def call_claude():
    messages = []
    for gpt_msg, claude_msg, llama_msg in zip(gpt_messages, claude_messages, llama_messages):
        messages.append({"role": "user", "content": gpt_msg})
        messages.append({"role": "assistant", "content": claude_msg})
        messages.append({"role": "user", "content": llama_msg})
                        
    # -- Debugging to see what messages are being passed
    #     print("Messages being sent to Claude:")
    # for idx, msg in enumerate(messages):
    #     print(f"{idx}: {msg}")
        
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [0]:
call_claude()

In [0]:
def call_ollama():
    messages = [{"role": "system", "content": llama_system}]
    for gpt_msg, claude_msg, llama_msg in zip(gpt_messages, claude_messages, llama_messages):
        messages.append({"role": "user", "content": gpt_msg})
        messages.append({"role": "user", "content": claude_msg})
        messages.append({"role": "assistant", "content": llama_msg})
    messages.append({"role": "user", "content": gpt_messages[-1]})

    try:
        response = ollama.chat(
            model=ollama_model,
            messages=messages
        )
        return response["message"]["content"]

    except Exception as e:
        print(f"Error in Llama call: {e}")
        return "An error occurred in Llama."

In [0]:
call_ollama()

In [0]:
print(f"\n{gpt_messages[0]}\n")
print(f"\n{claude_messages[0]}\n")
print(f"\n{llama_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    llama_next = call_ollama()
    print(f"Ollama:\n{llama_next}\n")
    llama_messages.append(llama_next)